In [1]:
from __future__ import print_function
import re

In [2]:
file = open("fullTest.txt", "r")
rawText = file.read()

In [3]:
p = re.compile(r"(?<=[.?!'\n])(\s+)(?=[A-Z'])")
ansText = p.sub('</s>\\1<s>', rawText)
# print(ansText)

In [4]:
dataset = []
labels = []
num_context = 11
dictionary = ['.', ',', ';', '!', '?', '\w', '\s', '\'', '[', ']', '-']
punctuations = ".?!"
tags = ["<s>", "</s>"]

def check_tag(text, index):
    if (index < 0):
        return -1
    if (index+4 < len(text) and text[index:index+4] in tags):
        return 1
    if (index+3 < len(text) and text[index:index+3] in tags):
        return 0
    return -1

def encoding(c):
    l = [0]*len(dictionary)
    for i in range(len(dictionary)):
        if dictionary[i] == c:
            l[i] = 1
            return l
    if c.isalpha():
        l[dictionary.index('\w')] = 1
    elif c.isspace():
        l[dictionary.index('\s')] = 1
    return l

for i in range(len(ansText)-3):
    if ansText[i] in punctuations:
        data = [[]]*num_context
        index = num_context//2
        data[index] = encoding(ansText[i])
#         print(ansText[i])
        index -= 1
        j = i - 1
        
        while index >= 0 and j >= 0:
            if check_tag(ansText, j-2) == 0:
                j -= 3
                continue
            if check_tag(ansText, j-3) == 1:
                j -= 4
                continue
            data[index] = encoding(ansText[j])
            index -= 1
            j -= 1
                
        index = num_context//2 + 1
        j = i + 1
        while index < num_context and j < len(ansText):
            if (check_tag(ansText, j) == -1):
                data[index] = encoding(ansText[j])
                index += 1
                j += 1
            else:
                j += len(tags[check_tag(ansText, j)])
                
        if ansText[i+1:i+3] == "</":
            labels.append(1)
        else:
            labels.append(0)
        dataset.append([item for sublist in data for item in sublist])
        
        
# print(sum(labels))
# print(dataset,labels)
# for i in range(len(labels)):
#     if labels[i] == 0:
#         print(dataset[i])
#         print('\n')

In [21]:
inputs = []
for i in range(len(dataset)):
    inputs_tmp = [0]*num_context*len(dictionary)
    for j in range(len(dataset[i])):
        inputs_tmp[j] = (dataset[i][j])
    inputs.append(inputs_tmp)
dataset = inputs
l = int(0.6*len(dataset))
dataset_train = dataset[:l]
dataset_test = dataset[l:]

labels_train = labels[:l]
labels_test = labels[l:]

In [16]:
from sklearn import linear_model
reg = linear_model.LinearRegression()

In [17]:
# print(dataset)
reg.fit(dataset_train, labels_train)
# reg.fit ([[0, 0], [1, 1], [2, 2]], [0, 1, 2])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [18]:
predicted_labels = reg.predict(dataset_test)

In [19]:
for i in range(len(predicted_labels)):
    if predicted_labels[i] > 0.5:
        predicted_labels[i] = 1
    else:
        predicted_labels[i] = 0
# print(sum(labels_test))
# print(predicted_labels - labels_test)
errors = 0
for i in range(len(predicted_labels)):
    errors += abs(predicted_labels[i] - labels_test[i])
print("Accuracy: %.2f" % ((sum(labels_test) - errors)*100/sum(labels_test)))

Accuracy: 98.82
